# Quantile delta mapping of maximum intensity



In [ ]:
%matplotlib inline

import os
from os.path import join as pjoin
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches

import re
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import cartopy.crs as ccrs
import cartopy.feature as feature
import statsmodels.api as sm
from datetime import timedelta, datetime

import shapely.geometry as sg
from shapely.geometry import box as sbox
from shapely.geometry import LineString, Point

from scipy.optimize import curve_fit
import scipy.stats as stats

# From TCRM codebase
from Utilities.loadData import maxWindSpeed

from builtins import str
sns.set_style('whitegrid')
sns.set_context("poster")

In [ ]:
def load_track_file(filename):
    """
    Load a TCLV file into a :class:`pandas.DataFrame`, and add a field 
    representing the age of each TCLV in hours, and the pressure difference.
    
    :param str filename: Path to a TCLV data file
    
    :returns: :class:`pandas.DataFrame`
    """
    # This assumes the format of the TCLV files is identical
    columns = ['num', 'year', 'month', 'day', 'hour', 'lon', 'lat',
               'pmin', 'vorticity', 'vmax', 'tanomsum','tanomdiff',
               'pmslanom', 'poci', 'reff','ravg','asym']
    df = pd.read_csv(filename, delimiter=' ', skipinitialspace=True,
                     names=columns, parse_dates={'datetime':[1,2,3,4]},
                     keep_date_col=True, 
                     dtype={'year':int, 'month':int, 'day':int})
    df['dt'] = df.groupby('num')['datetime'].apply(lambda x: x.diff())
    df['dt'] = df['dt'].transform(lambda x: x.total_seconds())

    df['age'] = df.groupby('num')['dt'].apply(np.cumsum).fillna(0)/3600.
    # Throw in the pressure deficit for good measure:
    df['pdiff'] = df['poci'] - df['pmin']
    # And normalised intensity. This is the intensity at any given time,
    # dividied by the lifetime maximum intensity for each unique event
    df['ni'] = df.pdiff / df.groupby('num').pdiff.transform(np.max)
    
    idx = df.groupby('num')['pdiff'].transform(max) == df['pdiff']
    return df[idx]

def filter_tracks(df, start_year=1980, end_year=2010, zeta=0, age=36):
    """
    Takes a `DataFrame` and filters on the basis of a prescribed vorticity 
    threshold, lifetime and a given time period.
    
    :param df: :class:`pandas.DataFrame` that holds the TCLV data
    :param int start_year: Starting year of the time period to filter
    :param int end_year: End year of the period to filter
    :param float zeta: Vorticity threshold to filter the TCLV data. 
                       This can be a positive value, as we filter on the
                       absolute value of the field.
    :param int age: Minimum age of the TCLVs in hours
    
    """
    tracks = df.groupby('num')
    filterdf = tracks.filter(lambda x: (x['datetime'].dt.year.min() >= start_year) &\
                                       (x['datetime'].dt.year.max() <= end_year) &\
                                       #(x['age'].max() >= age) &\
                                       (np.abs(x['vorticity'].min()) > zeta))
    return filterdf

def filter_tracks_domain(df, minlon=90, maxlon=180, minlat=-40, maxlat=0):
    """
    Takes a `DataFrame` and filters on the basis of whether the track interscts
    the given domain, which is specified by the minimum and maximum longitude and 
    latitude.
    
    NOTE: This assumes the tracks and bounding box are in the same geographic 
    coordinate system (i.e. generally a latitude-longitude coordinate system). 
    It will NOT support different projections (e.g. UTM data for the bounds and
    geographic for the tracks).
    
    NOTE: This doesn't work if there is only one point for the track. 
    
    :param df: :class:`pandas.DataFrame` that holds the TCLV data
    :param float minlon: minimum longitude of the bounding box
    :param float minlat: minimum latitude of the bounding box
    :param float maxlon: maximum longitude of the bounding box
    :param float maxlat: maximum latitude of the bounding box
    """
    domain = sbox(minlon, minlat, maxlon, maxlat, ccw=False)
    tracks = df.groupby('num')
    tempfilter = tracks.filter(lambda x: len(x) > 1)
    tempfilter.head()
    filterdf = tempfilter.groupby('num').filter(lambda x: LineString(zip(x['lon'], x['lat'])).intersects(domain))
    return filterdf

def filter_points_domain(df, minlon=90, maxlon=180, minlat=-40, maxlat=0):
    domain = sbox(minlon, minlat, maxlon, maxlat, ccw=False)
    filterdf = df[df.apply(lambda x: Point(x.lon, x.lat).within(domain), axis=1)]
    return filterdf

def calculateMaxWind(df, dtname='ISO_TIME'):
    """
    Calculate a maximum gust wind speed based on the central pressure deficit and the 
    wind-pressure relation defined in Holland (2008). This uses the function defined in 
    the TCRM code base, and simply passes the correct variables from the data frame
    to the function
    
    This returns a `DataFrame` with an additional column (`vmax`), which represents an estimated
    0.2 second maximum gust wind speed.
    """
    idx = df.num.values
    varidx = np.ones(len(idx))
    varidx[1:][idx[1:]==idx[:-1]] = 0
    
    dt = (df[dtname] - df[dtname].shift()).fillna(pd.Timedelta(seconds=0)).apply(lambda x: x / np.timedelta64(1,'h')).astype('int64') % (24*60)
    df['vmax'] = maxWindSpeed(varidx, dt.values, df.lon.values, df.lat.values,
                              df.pmin.values, df.poci.values, gustfactor=1.223)
    return df

In [ ]:
path = "../data/tclv/"
regex = r'all_tracks_(.+)_(rcp\d+)\.dat'
labels = ['TD', 'TC1', 'TC2', 'TC3', 'TC4', 'TC5']
catpal = sns.blend_palette([(0.000, 0.627, 0.235), (0.412, 0.627, 0.235), 
                            (0.663, 0.780, 0.282), (0.957, 0.812, 0.000), 
                            (0.925, 0.643, 0.016), (0.835, 0.314, 0.118),
                            (0.780, 0.086, 0.118)], 6)
dist = stats.lognorm

In [ ]:
best = pd.read_csv('../data/ibtracs.since1980.list.v04r00.csv', 
                       skiprows=[1],
                       usecols=[0,6,8,9,11,113],
                       na_values=[' '],
                       parse_dates=[1])
best.rename(columns={'SID':'num', 'LAT': 'lat', 'LON':'lon', 'WMO_PRES':'pmin', 'BOM_POCI':'poci'}, inplace=True)
best = best[best.poci.notnull() & best.pmin.notnull()]
best['pdiff'] = best.poci - best.pmin
best = best[best.pdiff > 0]
idx = best.groupby('num')['pdiff'].transform(max) == best['pdiff']
obstc = filter_points_domain(best[idx], 135, 160, -25, -10)

In [ ]:
def qdm(vobs, vref, vfut, dist=stats.lognorm):
    """
    Calculate the quantile delta mapping for a collection of simulated data. 
    
    This function is based on the formulation described in Cannon _et al._ (2015) and Heo _et al._ (2019).
    
    :param vobs: numpy.array of observed values
    :param vref: numpy.array of reference period values (simulated)
    :param vfut: numpy.array of future period values (simulated)
    :param func dist: Distribution function to use. This must have `fit`, `pdf`, `cdf` and `ppf` methods defined, 
                      as used in `scipy.stats.rv_continuous` distributions. Default is `scipy.stats.lognorm`.
                      
    :returns: `vfutb` a numpy.array of bias corrected future values. 
    """
    
    pobs = dist.fit(vobs, loc=0, scale=1)
    pref = dist.fit(vref, loc=0, scale=1)
    pfut = dist.fit(vfut, loc=0, scale=1)
    
    # CDF of future, at the value of the future data points
    Fsf = dist.cdf(vfut, *pfut)
    # Inverse cdf of reference period distribution, evaluated at future CDF
    # values
    invFsr = dist.ppf(Fsf, *pref)
    
    # Relative change in values
    delta = vfut / invFsr
    vfutb = dist.ppf(Fsf, *pobs) * delta
    
    return vfutb

In [ ]:
refdata = {}
regex = r'all_tracks_(.+)_(rcp\d+)\.dat'
ens45 = []
ens85 = []
for i, fname in enumerate(os.listdir(path)):
    if fname=="all_tracks_ERAIntQ_rcp85.dat":
        continue
    m = re.match(regex, fname)
    try:
        model, rcp = m.group(1, 2)
    except AttributeError: 
        # Catch the case where there are other files or folders in the directory
        continue
    
    filename = pjoin(path, fname)
    df = load_track_file(filename)
    df = filter_tracks(df)
    df = filter_points_domain(df, 135, 160, -25, -10)
    if rcp.upper() == 'RCP45':
        ens45.append(df)
    else:
        ens85.append(df)
    label = f"{model} {rcp.upper()}"
    refdata[label] = df

In [ ]:
refdata['ENS RCP45'] = pd.concat(ens45, ignore_index=True)
refdata['ENS RCP85'] = pd.concat(ens85, ignore_index=True)

In [ ]:
fig, axes = plt.subplots(6,4, figsize=(20,20), sharex=True)
ax = axes.flatten()
bins = np.arange(0, 100, 5)
refparams = pd.DataFrame(columns=['Model', 'RCP', 'mu', 'sigma', 'zeta'])
for i, (m, df) in enumerate(refdata.items()):
    sns.distplot(df.pdiff, ax=ax[i], kde=False, norm_hist=True)
    model, rcp = m.split(' ')
    popt = stats.lognorm.fit(df.pdiff, loc=0, scale=1)
    refparams = refparams.append({'Model':model, 'RCP':rcp, 
                                  'mu':popt[0], 'sigma':popt[1], 'zeta':popt[2]},
                                ignore_index=True)
    fitline = stats.lognorm.pdf(np.arange(0, 101), *popt, )
    ax[i].plot(np.arange(0,101), fitline, color='r')
    ax[i].set_title("{0}\n({1:.4f}, {2:.4f}, {3:.4f})".format(m, *popt))
 

# Add the IBTrACS as reference
#sns.distplot(obstc.pdiff, fit=stats.lognorm, kde=False, color='red', ax=ax[-1])
obsparams = stats.lognorm.fit(obstc.pdiff, loc=0, scale=1)
#ax[-1].set_title("IBTrACS\n({0:.4f}, {1:.4f}, {2:.4f})".format(*obsparams))
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,5), sharey=True)
x = np.arange(0, 100)

for index, row in refparams[refparams['RCP']=='RCP45'].iterrows():
    popt = (row.mu, row.sigma, row.zeta)
    fitline = stats.lognorm.pdf(x, *popt)
    if row.Model == 'ENS':
        ax[0].plot(x, fitline, color='b')
    else:
        ax[0].plot(x, fitline, color='k')
        
ax[0].plot(x, stats.lognorm.pdf(x, *obsparams), 'r')
ax[0].set_title("RCP4.5")


for index, row in refparams[refparams['RCP']=='RCP85'].iterrows():
    popt = (row.mu, row.sigma, row.zeta)
    fitline = stats.lognorm.pdf(x, *popt)
    if index == 1:
        ax[1].plot(x, fitline, color='k', label='Reference')
    elif row.Model == 'ENS':
        ax[1].plot(x, fitline, color='b', label='Ensemble')
    else:
        ax[1].plot(x, fitline, color='k')
        
ax[1].plot(x, stats.lognorm.pdf(x, *obsparams), 'r', label='Observations')
ax[1].set_title("RCP8.5")
fig.tight_layout()
ax[0].set_xlabel(r"$\Delta p_c$ (hPa)")
ax[0].set_ylabel("Probability")
ax[1].set_xlabel(r"$\Delta p_c$ (hPa)")
ax[1].legend()
None

In [ ]:
start=2081
end=2100
futdata = {}
ens45 = []
ens85 = []
for i, f in enumerate(os.listdir(path)):
    # Exclude the ERA Interim data
    if f=="all_tracks_ERAIntQ_rcp85.dat":
        continue
    m = re.match(regex, f)
    try:
        model, rcp = m.group(1, 2)
    except AttributeError:
        continue
    
    filename = pjoin(path, f)
    df = load_track_file(filename)
    df = filter_tracks(df, start_year=start, end_year=end)
    df = filter_points_domain(df, 135, 160, -25, -10)
    if rcp.upper() == 'RCP45':
        ens45.append(df)
    else:
        ens85.append(df)
    label = "{0} {1}".format(model, rcp.upper())
    futdata[label] = df
    
futdata['ENS RCP45'] = pd.concat(ens45, ignore_index=True)
futdata['ENS RCP85'] = pd.concat(ens85, ignore_index=True)

In [ ]:
fig, axes = plt.subplots(6,4, figsize=(20,20), sharex=True)
ax = axes.flatten()
futparams = pd.DataFrame(columns=['Model', 'RCP', 'mu', 'sigma', 'zeta'])
for i, (m, df) in enumerate(futdata.items()):
    model, rcp = m.split(' ')
    sns.distplot(df.pdiff, ax=ax[i], kde=False, norm_hist=True)
    popt = stats.lognorm.fit(df.pdiff, loc=0, scale=1)
    fitline = stats.lognorm.pdf(np.arange(0, 101), *popt, )
    futparams = futparams.append({'Model':model, 'RCP':rcp,
                                  'mu':popt[0], 'sigma':popt[1], 'zeta':popt[2]},
                                  ignore_index=True)
    ax[i].plot(np.arange(0,101), fitline, color='r')
    ax[i].set_title("{0}\n({1:.4f}, {2:.4f}, {3:.4f})".format(m, *popt))
 

# Again, add the IBTrACS as reference
#sns.distplot(obstc.pdiff, fit=stats.lognorm, kde=False, color='red', ax=ax[-1])
obsparams = stats.lognorm.fit(obstc.pdiff)
#ax[-1].set_title("IBTrACS\n({0:.4f}, {1:.4f}, {2:.4f})".format(*obsparams))
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,6), sharey=True)
x = np.arange(0, 100)

for index, row in futparams[refparams['RCP']=='RCP45'].iterrows():
    popt = (row.mu, row.sigma, row.zeta)
    fitline = stats.lognorm.pdf(x, *popt)
    if row.Model == 'ENS':
        ax[0].plot(x, fitline, color='b')
    else:
        ax[0].plot(x, fitline, color='k')
    
ax[0].plot(x, stats.lognorm.pdf(x, *obsparams), 'r')
ax[0].set_title("RCP4.5")


for index, row in futparams[refparams['RCP']=='RCP85'].iterrows():
    popt = (row.mu, row.sigma, row.zeta)
    fitline = stats.lognorm.pdf(x, *popt)
    if index == 1:
        ax[1].plot(x, fitline, color='k', label='Projected')
    elif row.Model == 'ENS':
        ax[1].plot(x, fitline, color='b', label='Ensemble')
    else:
        ax[1].plot(x, fitline, color='k')
        
ax[1].plot(x, stats.lognorm.pdf(x, *obsparams), 'r', label='Observations')
ax[1].set_title("RCP8.5")
fig.tight_layout()
ax[0].set_xlabel(r"$\Delta p_c$ (hPa)")
ax[1].set_xlabel(r"$\Delta p_c$ (hPa)")
ax[1].legend()
None

In [ ]:
fig, axes = plt.subplots(6, 4, figsize=(20, 20), sharex=True, sharey=True)
ax = axes.flatten()
x = np.linspace(0, 1, 101)
for i, (m, refdf) in enumerate(refdata.items()):
    futdf = futdata[m]
    model, rcp = m.split(' ')

    srefdata = refdf.pdiff.values
    sfutdata = futdf.pdiff.values
    srefq = np.quantile(srefdata, x)
    sfutq = np.quantile(sfutdata, x)
    ax[i].scatter(srefq, sfutq, alpha=0.5)
    ax[i].plot([0, 50], [0,50], '--', color='k', alpha=0.5)
    ax[i].set_title(m, size='small')
    ax[i].set_xlim((0, 50))
    ax[i].set_ylim((0, 50))
    ax[i].set_aspect('equal')
    
fig.tight_layout()
fig.text(0.5, 0.01, "Reference quantiles (hPa)", ha='center', va='center')
fig.text(0.01, 0.5, "Future quantiles (hPa)", ha='center', va='center', rotation='vertical')
None

In [ ]:
refdf = refdata['CSIRO-Mk3-6-0Q RCP45']
futdf = futdata['CSIRO-Mk3-6-0Q RCP45']

refq099 = np.quantile(refdf.pdiff, 0.99)
futq099 = np.quantile(futdf.pdiff, 0.99)
obsq099 = np.quantile(obstc.pdiff, 0.99)
delta = futq099/refq099 
print(delta)
fig, ax = plt.subplots(1, 2, figsize=(16, 8), sharey=True)
ax[0].scatter(refdf.datetime, refdf.pdiff, s=1, c='r')
ax[0].scatter(futdf.datetime, futdf.pdiff, s=1, c='r')
ax[0].set_xlim(datetime(1981, 1, 1), datetime(2100, 1, 1))
ax[0].axhline(refq099, xmin=0, xmax=0.833333, linestyle='--', color='0.5')
ax[0].axhline(refq099, xmin=0, xmax=0.25, linestyle='--', color='k')
ax[0].axhline(futq099, xmin=0.83333, xmax=1, linestyle='--', color='k')
ax[1].scatter(obstc.ISO_TIME, obstc.pdiff, s=1, c='b')
ax[1].axhline(obsq099, xmin=0, xmax=0.83333, linestyle='--', color='0.5')
ax[1].axhline(obsq099, xmin=0, xmax=0.3333, linestyle='--', color='k')
ax[1].axhline(obsq099*delta, xmin = 0.83333, xmax=1, linestyle='--', color='k')
ax[1].set_xlim(datetime(1981, 1, 1), datetime(2100, 1, 1))
ax[0].set_ylabel(r"$\Delta p_{lmi}$ [hPa]")
ax[0].set_xlabel("Year")
ax[1].set_xlabel("Year")
ax[0].set_title('Modelled')
ax[1].set_title('Observed')

styles = mpatches.ArrowStyle.get_styles()
def to_texstring(s):
    s = s.replace("<", r"$<$")
    s = s.replace(">", r"$>$")
    s = s.replace("|", r"$|$")
    return s

x=datetime(2080, 1, 1)
y = 0.5 * (refq099 + futq099)
tx=datetime(2060, 1, 1)
ty=0.5 * (refq099 + futq099)
stylename = '-['
ax[0].annotate(r'$\delta_{fut}$' + ' = {0:.2f}'.format(delta), (x, y),
               (tx, ty), xycoords='data',
                ha="right", va="center",
                size=1.4,
                arrowprops=dict(arrowstyle=stylename,
                                fc="b", ec="b",
                                connectionstyle="arc3,rad=-0.05",
                                ),
              fontsize='x-small', color='b',
              bbox=dict(boxstyle="square", fc="w", alpha=0.5))

ax[0].annotate(r"$Q_{ref}(0.99)$" + " = {0:.1f}".format(refq099), 
               (datetime(1995, 1, 1), refq099),
               (datetime(2000, 1, 1), 3*refq099),
               ha="center", va="center", size=1.4,
               arrowprops=dict(arrowstyle='->',
                                fc="b", ec="b", shrinkB=1.5,
                                connectionstyle="arc3,rad=-0.0",
                                ),
               fontsize='x-small', color='b',
              bbox=dict(boxstyle="square", fc="w", alpha=0.5))

ax[0].annotate(r"$Q_{fut}(0.99)$" + " = {0:.1f}".format(futq099), 
               (datetime(2090, 1, 1), futq099),
               (datetime(2080, 1, 1), 2*futq099),
               ha="center", va="center", size=1.4,
               arrowprops=dict(arrowstyle='->',
                                fc="b", ec="b", shrinkB=1.5,
                                connectionstyle="arc3,rad=-0.0",
                                ),
               fontsize='x-small', color='b',
              bbox=dict(boxstyle="square", fc="w", alpha=0.5))

ax[1].annotate(r"$Q_{obs}(0.99)$" + " = {0:.1f}".format(obsq099),
              (datetime(1995, 1, 1), obsq099),
              (datetime(1995, 1, 1), 1.5*obsq099), 
              ha="center", va='center', size=1.4,
              arrowprops=dict(arrowstyle="->",
                             fc='b', ec='b', shrinkB=1.5,
                             connectionstyle='arc3,rad=-0.05'),
              fontsize='x-small', color='b',
              bbox=dict(boxstyle="square", fc="w", alpha=0.5))

ax[1].annotate(r"$Q_{futb}(0.99)$" + " = {0:.1f}".format(obsq099 * delta),
              (datetime(2090, 1, 1), obsq099 * delta),
              (datetime(2060, 1, 1), 1.5*obsq099), 
              ha="center", va='center', size=1.4,
              arrowprops=dict(arrowstyle="->",
                             fc='b', ec='b', shrinkB=1.5,
                             connectionstyle='arc3,rad=0.5'),
              fontsize='x-small', color='b',
              bbox=dict(boxstyle="square", fc="w", alpha=0.5))

In [ ]:
obsdata = obstc.pdiff.values[obstc.pdiff.values > 0]
fig, axes = plt.subplots(6,4, figsize=(20,20), sharex=True)
ax = axes.flatten()
brefdata = {}
brefparams = pd.DataFrame(columns=['Model', 'RCP', 'mu', 'sigma', 'zeta'])
for i, (m, refdf) in enumerate(refdata.items()):
    futdf = futdata[m]
    model, rcp = m.split(' ')

    srefdata = refdf.pdiff.values[refdf.pdiff.values > 0]
    brefdata[m] = qdm(obsdata, srefdata, srefdata)
    
    popt = stats.lognorm.fit(brefdata[m], loc=0, scale=1)
    brefparams = brefparams.append({'Model':model, 'RCP':rcp,
                                    'mu':popt[0], 'sigma':popt[1], 'zeta':popt[2]},
                                   ignore_index=True)
    ax[i].scatter(srefdata, brefdata[m], alpha=0.5)
    ax[i].plot([-5, 100], [-5, 100], '--')
    ax[i].set_xlabel("raw-tclv (hPa)")
    ax[i].set_ylabel("corrected-tclv (hPa)")
    ax[i].set_title(m)
    ax[i].set_xlim((-5, 100))
    ax[i].set_ylim((-5, 200))
    
fig.tight_layout()

In [ ]:
obsdata = obstc.pdiff.values[obstc.pdiff.values > 0]
fig, axes = plt.subplots(6,4, figsize=(20,20), sharex=True, sharey=True)
ax = axes.flatten()
bfutparams = pd.DataFrame(columns=['Model', 'RCP', 'mu', 'sigma', 'zeta'])
bfutdata = {}
for i, (m, refdf) in enumerate(refdata.items()):
    futdf = futdata[m]
    model, rcp = m.split(' ')
    srefdata = refdf.pdiff.values[refdf.pdiff.values > 0]
    sfutdata = futdf.pdiff.values[futdf.pdiff.values > 0]
    bfutdata[m] = qdm(obsdata, srefdata, sfutdata)
    popt = stats.lognorm.fit(bfutdata[m], loc=0, scale=1)
    bfutparams = bfutparams.append({'Model':model, 'RCP':rcp,
                                    'mu':popt[0], 'sigma':popt[1], 'zeta':popt[2]},
                                   ignore_index=True)
    ax[i].scatter(sfutdata, bfutdata[m], alpha=0.5)
    ax[i].plot([-5, 100], [-5, 100], '--')
    ax[i].set_xlabel("raw-tclv (hPa)")
    ax[i].set_ylabel("corrected-tclv (hPa)")
    ax[i].set_title(m)
    ax[i].set_xlim((-5, 100))
    ax[i].set_ylim((-5, 200))

In [ ]:
fig, axes = plt.subplots(6, 4, figsize=(20, 20), sharex=True, sharey=True)
ax = axes.flatten()

x = np.arange(0, 51)
for i, row in enumerate(refparams.itertuples()):
    m = row.Model
    r = row.RCP
    refpopt = (row.mu, row.sigma, row.zeta)
    futrow = futparams[(futparams.Model==m) & (futparams.RCP==r)]
    futpopt = (futrow.mu, futrow.sigma, futrow.zeta)
    refpdf = stats.lognorm.pdf(x, *refpopt)
    futpdf = stats.lognorm.pdf(x, *futpopt)
    ks, pv = stats.ks_2samp(refpdf, futpdf)
    ax[i].plot(x, refpdf, 'k', label='Reference')
    ax[i].plot(x, futpdf, 'r', label='Future')
    ax[i].plot(x, stats.lognorm.pdf(x, *obsparams), 'g', lw=1, label='IBTrACS')
    ax[i].set_title('{0} {1}\n({2:.4f} {3:.4f})'.format(m, r, ks, pv))
    if i==0: ax[i].legend(loc=1)
        
#ax[-1].plot(x, stats.lognorm.pdf(x, *obsparams))
#ax[-1].set_title("IBTrACS")
fig.tight_layout()

In [ ]:
fig, axes = plt.subplots(6, 4, figsize=(20, 20), sharex=True, sharey=True)
ax = axes.flatten()

x = np.arange(0, 101)
for i, row in enumerate(brefparams.itertuples()):
    m = row.Model
    r = row.RCP
    refpopt = (row.mu, row.sigma, row.zeta)
    futrow = bfutparams[(bfutparams.Model==m) & (bfutparams.RCP==r)]
    futpopt = (futrow.mu, futrow.sigma, futrow.zeta)
    refpdf = stats.lognorm.pdf(x, *refpopt)
    futpdf = stats.lognorm.pdf(x, *futpopt)
    ks, pv = stats.ks_2samp(refpdf, futpdf)
    ax[i].plot(x, refpdf, 'k', label='Reference')
    ax[i].plot(x, futpdf, 'r', label='Future')
    ax[i].plot(x, stats.lognorm.pdf(x, *obsparams), 'g', lw=1, label='IBTrACS')
    ax[i].set_title('{0} {1}\n({2:.4f} {3:.4f})'.format(m, r, ks, pv))
    if i==0: ax[i].legend(loc=1 )
        
#ax[-1].plot(x, stats.lognorm.pdf(x, *obsparams))
#ax[-1].set_title("IBTrACS")
fig.tight_layout()
None

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,5), sharey=True)
x = np.arange(0, 101, 0.5)
for i, row in enumerate(brefparams.itertuples()):
    refpopt = (row.mu, row.sigma, row.zeta)
    refpdf = stats.lognorm.pdf(x, *refpopt)
    if i == 0:
        ax[0].plot(x, refpdf, 'k', label="Reference", alpha=0.5)
    else:
        ax[0].plot(x, refpdf, 'k', alpha=0.5)

for i, row in enumerate(bfutparams.itertuples()):
    futpopt =(row.mu, row.sigma, row.zeta)
    futpdf = stats.lognorm.pdf(x, *futpopt)
    if i == 0:
        ax[1].plot(x, futpdf, 'k', label="Projected", alpha=0.5)
    else:
        ax[1].plot(x, futpdf, 'k', alpha=0.5)
    
ax[0].plot(x, stats.lognorm.pdf(x, *obsparams), 'r', label='IBTrACS')
ax[1].plot(x, stats.lognorm.pdf(x, *obsparams), 'r', label='IBTrACS')    
ax[0].legend(loc=1)
ax[1].legend(loc=1)

ax[0].set_xlabel(r'$\Delta p_c$ (hPa)')
ax[1].set_xlabel(r'$\Delta p_c$ (hPa)')
ax[0].set_ylabel("Probability")
#ax[1].set_ylabel("Probability")
fig.tight_layout()
None

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,6), sharey=True)
x = np.arange(0, 100)

for index, row in bfutparams[refparams['RCP']=='RCP45'].iterrows():
    popt = (row.mu, row.sigma, row.zeta)
    fitline = stats.lognorm.pdf(x, *popt)
    ax[0].plot(x, fitline, color='k')
ax[0].plot(x, stats.lognorm.pdf(x, *obsparams), 'r')
ax[0].set_title("RCP4.5")


for index, row in bfutparams[refparams['RCP']=='RCP85'].iterrows():
    popt = (row.mu, row.sigma, row.zeta)
    fitline = stats.lognorm.pdf(x, *popt)
    if index == 1:
        ax[1].plot(x, fitline, color='k', label='Projected')
    else:
        ax[1].plot(x, fitline, color='k')
ax[1].plot(x, stats.lognorm.pdf(x, *obsparams), 'r', label='Observations')
ax[1].set_title("RCP8.5")
fig.tight_layout()
ax[0].set_xlabel(r"$\Delta p_c$ (hPa)")
ax[1].set_xlabel(r"$\Delta p_c$ (hPa)")
ax[1].legend()
None